In [ ]:
!java -version

In [ ]:
import pandas as pd
import numpy as np
import tabula
from tabulate import tabulate
import re


In [ ]:
bach_cols_2021 = [
    'cip_code',
    'total_graduates',
    'knowledge_rate',
    'career_outcomes_perc',
    'total_perc_employed_overall',
    'total_perc_employed_full_time',
    'total_perc_employed_part_time',
    'perc_standard_employment_overall',
    'perc_standard_full_time',
    'perc_standard_part_time',
    'perc_entrepreneur_overall',
    'perc_entrepreneur_full_time',
    'perc_entrepreneur_part_time',
    'perc_temp_contract_overall',
    'perc_temp_contract_full_time',
    'perc_temp_contract_part_time',
    'perc_freelance_overall',
    'perc_freelance_full_time',
    'perc_freelance_part_time',
    'perc_fellowship_intern_overall',
    'perc_fellowship_intern_full_time',
    'perc_fellowship_intern_part_time',
    'perc_service',
    'perc_military',
    'perc_continuing_ed',
    'perc_still_seeking_overall',
    'perc_seeking_employment',
    'perc_seeking_continuing_ed',
    'perc_not_seeking',
    'mean_starting_salary'
    ]

mast_cols_2021 = [
    'cip_code',
    'total_graduates',
    'knowledge_rate',
    'career_outcomes_perc',
    'total_perc_employed_overall',
    'total_perc_employed_full_time',
    'total_perc_employed_part_time',
    'perc_standard_employment_overall',
    'perc_standard_full_time',
    'perc_standard_part_time',
    'perc_faculty_overall',
    'perc_faculty_tenure',
    'perc_faculty_non_tenure',
    'perc_entrepreneur_overall',
    'perc_entrepreneur_full_time',
    'perc_entrepreneur_part_time',
    'perc_temp_contract_overall',
    'perc_temp_contract_full_time',
    'perc_temp_contract_part_time',
    'perc_freelance_overall',
    'perc_freelance_full_time',
    'perc_freelance_part_time',
    'perc_fellowship_intern_overall',
    'perc_fellowship_intern_full_time',
    'perc_fellowship_intern_part_time',
    'perc_service',
    'perc_military',
    'perc_continuing_ed',
    'perc_still_seeking_overall',
    'perc_seeking_employment',
    'perc_seeking_continuing_ed',
    'perc_not_seeking',
    'mean_starting_salary'
    ]

In [387]:
bach_cols_2018 = list(np.append(bach_cols_2021[:-5], bach_cols_2021[-4:]))
bach_cols_2017 = bach_cols_2018
bach_cols_2016 = bach_cols_2018

mast_cols_2016 = list(np.append(mast_cols_2021[:10], mast_cols_2021[13:]))
mast_cols_2016 = list(np.append(mast_cols_2016[:-5], mast_cols_2016[-4:]))

In [ ]:
print(len(bach_cols_2017))

In [ ]:
bach_2021 = tabula.read_pdf('pdf_data\\2021.pdf', pages='16-73')
mast_2021 = tabula.read_pdf('pdf_data\\2021.pdf', pages='78-125')
#bach_2020 = tabula.read_pdf('pdf_data\\2020.pdf', pages='17-67')
#mast_2020 = tabula.read_pdf('pdf_data\\2020.pdf', pages='72-117')
bach_2018 = tabula.read_pdf('pdf_data\\2018.pdf', pages='24-71')
mast_2018 = tabula.read_pdf('pdf_data\\2018.pdf', pages='73-116')
bach_2017 = tabula.read_pdf('pdf_data\\2017.pdf', pages='25-70')
mast_2017 = tabula.read_pdf('pdf_data\\2017.pdf', pages='72-114')
bach_2016 = tabula.read_pdf('pdf_data\\2016.pdf', pages='25-70')
mast_2016 = tabula.read_pdf('pdf_data\\2016.pdf', pages='72-115')

In [388]:
bach_dict = {'2016':(bach_2016, bach_cols_2016), '2017':(bach_2017, bach_cols_2017), '2018':(bach_2018, bach_cols_2018), '2021':(bach_2021, bach_cols_2021)}
mast_dict = {'2016':(mast_2016, mast_cols_2016), '2017':(mast_2017, mast_cols_2021), '2018':(mast_2018, mast_cols_2021), '2021':(mast_2021, mast_cols_2021)}

In [364]:
def clean_and_combine(df_list, year, cols):
    #create empty df to combine into
    total_df = pd.DataFrame()

    #initial cleaning, get rid of unwanted empty columns
    new_df_list = []
    for i in df_list:
        df = i
        if 'Unnamed: 1' in df.columns:
            df = df.drop(columns='Unnamed: 1')
        if 'Unnamed: 2' in df.columns:
            df = df.drop(columns='Unnamed: 2')
        if 'Unnamed: 3' in df.columns:
            df = df.drop(columns='Unnamed: 3')
        if (len(df) > 1) and (len(df.columns) > 3):
            new_df_list.append(df[(df[df.columns[-2]].notna())&(df[df.columns[-3]].notna())])
        elif (len(df) > 1) and (len(df.columns) <= 3):
            new_df_list.append(df[df[df.columns[-2]].notna()])

    #shift misplaced rows
    middle_df_list = []
    for i in new_df_list:
        df = i.reset_index(drop=True)
        for ind,r in df.iterrows():
            if type(df.at[ind, df.columns[-1]]) == float:
                row_vals = ['NA']
                row_vals.extend(list(df.iloc[ind][:-1]))
                df.iloc[ind] = row_vals
        if len(df) > 3:
            middle_df_list.append(df)

    #split unnecessarily combined columns
    final_df_list = []
    for i in middle_df_list:
        df = i
        if type(df.at[0, df.columns[-1]]) == float:
            row_vals = ['NA']
            row_vals.extend(list(df.iloc[0][:-1]))
            df.iloc[0] = row_vals
        for x in df.columns:
            if ('Unnamed' not in x) and (' ' in x):
                col_to_sep = x
                col_sep = x.split(' ')
                df[col_sep] = df[col_to_sep].str.split(' ',n=1,expand=True)
                df = df.drop(columns=col_to_sep)
        final_df_list.append(df)

    #create copy of each df
    #shift columns if needed, transpose, and combine total_df
    for i in final_df_list:
        if len(i) >0:
            copy = i.copy()
            if copy.columns[0] != 'Unnamed: 0':
                columns = copy.columns[:-1].insert(0, 'Unnamed: 0')
                copy = copy.set_axis(columns,axis=1)
            copy = copy.iloc[-(len(cols)-1):].drop(columns='Unnamed: 0').T.reset_index()
            copy = copy.set_axis(cols, axis=1)
            total_df = pd.concat([total_df, copy], ignore_index=True)
    total_df['year'] = year

    #drop rows with null values
    total_df = total_df.dropna()

    #clean data and change dtypes to numeric
    total_df = total_df.loc[total_df['mean_starting_salary'] !='NA']
    total_grads = []
    for i in total_df['total_graduates'].values:
        try:
            total_grads.append(int(re.sub('[^1-9]', '', i)))
        except:
            total_grads.append(0)
    total_df['total_graduates'] = total_grads
    mean_sals = []
    for i in total_df['mean_starting_salary'].values:
        try:
            mean_sals.append(int(re.sub('[^1-9]', '', i)))
        except:
            mean_sals.append(re.sub('[^1-9]', '', i))
    total_df['mean_starting_salary'] = mean_sals
    for i in total_df.columns[2:-2]:
        col_vals = []
        for x in total_df[i].values:
            try:
                col_vals.append(float(re.sub('[^1-9.]','',x)))
            except:
                col_vals.append(float(0))
        total_df[i] = col_vals

    #return cleaned and combined df
    return total_df

In [376]:
mast_cols_2016

['cip_code',
 'total_graduates',
 'knowledge_rate',
 'career_outcomes_perc',
 'total_perc_employed_overall',
 'total_perc_employed_full_time',
 'total_perc_employed_part_time',
 'perc_standard_employment_overall',
 'perc_standard_full_time',
 'perc_standard_part_time',
 'perc_faculty_overall',
 'perc_faculty_tenure',
 'perc_faculty_non_tenure',
 'perc_entrepreneur_overall',
 'perc_entrepreneur_full_time',
 'perc_entrepreneur_part_time',
 'perc_temp_contract_overall',
 'perc_temp_contract_full_time',
 'perc_temp_contract_part_time',
 'perc_freelance_overall',
 'perc_freelance_full_time',
 'perc_freelance_part_time',
 'perc_fellowship_intern_overall',
 'perc_fellowship_intern_full_time',
 'perc_fellowship_intern_part_time',
 'perc_service',
 'perc_military',
 'perc_continuing_ed',
 'perc_seeking_employment',
 'perc_seeking_continuing_ed',
 'perc_not_seeking',
 'mean_starting_salary']

In [383]:
test_2016 = clean_and_combine(mast_2021, 2017, mast_cols_2021)

In [391]:
def get_all_data(df_dict, grad_threshold, kr_threshold):
    #create empty dataframe to combine into
    combined = pd.DataFrame()

    #add all desired years
    for i in df_dict.keys():
        year_df = clean_and_combine(df_dict[i][0], int(i), df_dict[i][1])
        combined = pd.concat([combined, year_df], ignore_index=True)

    #only keep data that meets desired graduate and knowledge rate criteria
    combined = combined.loc[(combined['total_graduates'] >= grad_threshold) & (combined['knowledge_rate'] >= kr_threshold)].reset_index(drop=True)
    return combined

In [392]:
all_bach = get_all_data(bach_dict, 200, 60)
all_mast = get_all_data(mast_dict, 200, 60)

In [393]:
all_bach

,cip_code,total_graduates,knowledge_rate,career_outcomes_perc,total_perc_employed_overall,total_perc_employed_full_time,total_perc_employed_part_time,perc_standard_employment_overall,perc_standard_full_time,perc_standard_part_time,...,perc_fellowship_intern_part_time,perc_service,perc_military,perc_continuing_ed,perc_seeking_employment,perc_seeking_continuing_ed,perc_not_seeking,mean_starting_salary,year,perc_still_seeking_overall
0,01,6688,63.0,83.8,64.1,6.7,3.3,56.7,54.2,2.4,...,0.3,0.2,0.2,18.6,12.2,3.8,1.0,451,2016,NaN
1,01.01,1618,74.8,86.9,75.9,72.9,3.0,68.8,66.8,2.1,...,0.2,0.1,0.2,9.9,11.1,1.9,0.9,45862,2016,NaN
2,01.08,228,82.0,83.9,64.2,59.9,4.3,58.8,55.6,3.2,...,0.0,0.5,0.0,18.7,14.4,1.6,0.5,36268,2016,NaN
3,01.09,1758,67.6,85.2,54.9,5.8,4.0,47.8,44.8,3.0,...,0.7,0.0,0.1,29.2,1.0,4.6,1.2,3674,2016,NaN
4,03,4882,62.8,76.2,6.1,51.3,8.8,5.1,43.7,6.5,...,0.7,2.3,1.1,11.3,17.8,5.5,1.9,36449,2016,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391,52.12,325,74.0,87.6,76.2,74.3,2.0,74.1,72.5,1.7,...,0.0,0.2,0.7,9.6,11.4,0.8,1.1,64734,2021,12.2
392,52.13,2764,76.6,89.0,76.1,74.4,1.7,69.7,68.4,1.3,...,0.2,0.0,0.3,11.9,9.9,1.0,0.8,65843,2021,1.9
393,52.14,1413,72.1,86.5,76.6,71.7,4.9,7.9,67.3,3.6,...,0.7,0.1,0.3,8.8,12.3,1.0,0.8,49481,2021,13.3
394,52.17,314,85.7,87.2,76.2,7.3,5.9,74.3,68.8,5.6,...,0.0,0.0,0.4,9.3,1.8,1.9,1.5,51463,2021,12.6


In [394]:
all_mast

,cip_code,total_graduates,knowledge_rate,career_outcomes_perc,total_perc_employed_overall,total_perc_employed_full_time,total_perc_employed_part_time,perc_standard_employment_overall,perc_standard_full_time,perc_standard_part_time,...,perc_continuing_ed,perc_seeking_employment,perc_seeking_continuing_ed,perc_not_seeking,mean_starting_salary,year,perc_faculty_overall,perc_faculty_tenure,perc_faculty_non_tenure,perc_still_seeking_overall
0,09,238,62.2,95.9,83.8,78.7,5.1,75.5,72.2,3.3,...,0.4,0.3,3.3,1.3,53915,2016,NaN,NaN,NaN,NaN
1,09.04,487,62.6,1.0,84.6,77.4,7.2,77.4,71.5,5.9,...,1.0,0.0,0.0,12.8,44676,2016,NaN,NaN,NaN,NaN
2,11,679,65.4,96.4,86.1,84.3,1.8,82.3,81.1,1.2,...,0.0,0.2,3.0,9.7,81619,2016,NaN,NaN,NaN,NaN
3,11.07,344,65.1,96.4,84.9,83.9,1.0,82.1,81.4,0.7,...,0.0,0.1,3.2,1.5,76934,2016,NaN,NaN,NaN,NaN
4,14,1449,63.4,8.9,65.3,64.1,1.2,63.1,62.3,0.8,...,0.3,2.3,13.4,17.4,78866,2016,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,52.03,2658,71.5,95.5,89.2,87.7,1.5,86.6,85.2,1.4,...,4.8,4.2,0.2,1.2,5873,2021,0.1,0.1,0.1,4.4
113,52.08,763,69.6,88.1,78.2,77.0,1.1,73.6,72.7,0.9,...,7.9,11.3,0.6,0.8,7224,2021,0.8,0.2,0.6,11.9
114,52.10,1425,63.6,9.6,83.4,82.0,1.3,79.9,78.9,1.0,...,4.6,8.8,0.4,1.0,72,2021,0.2,0.1,0.1,9.3
115,52.14,265,73.6,86.4,74.4,71.3,3.1,64.1,63.1,1.0,...,1.3,12.8,0.0,2.6,58611,2021,0.0,0.0,0.0,12.8
